In [1]:
import pandas as pd
import sqlite3
import os
import re

In [2]:
def split_into_sentences(text):
    
    sentence_endings = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\!|\?)\s')
    sentences = sentence_endings.split(text)    

    sentences = [sentence.strip() for sentence in sentences if sentence]
    
    return sentences

In [3]:
conn = sqlite3.connect("etl.sqlite3")

In [4]:
cursor = conn.cursor()

In [5]:
# Crear la tabla documento
cursor.execute('''
    CREATE TABLE IF NOT EXISTS documento (
        id_documento INTEGER PRIMARY KEY,
        filename TEXT,
        text TEXT
    )
''')

In [6]:
# Confirmar cambios
conn.commit()

In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS parrafo (
        id_parrafo INTEGER PRIMARY KEY,
        parrafo_texto TEXT,
        parrafo_limpio TEXT,
        init_p INTEGER,
        length_p INTEGER,
        id_documento INTEGER,
        FOREIGN KEY (id_documento) REFERENCES documento(id_documento)
    )
''')

In [8]:
conn.commit()

In [9]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS oracion (
        id_oracion INTEGER PRIMARY KEY,
        oracion_texto TEXT,
        oracion_limpio TEXT,
        init_o INTEGER,
        length_o INTEGER,
        id_parrafo INTEGER,
        FOREIGN KEY (id_parrafo) REFERENCES documento(id_parrafo)
    )
''')

In [10]:
conn.commit()

In [11]:
df = pd.read_excel("intrinsic.xlsx", sheet_name="Sheet1")
# Si el excel tiene una sola hoja, no hace falta pasar el parametro sheet_name
# en caso contrario, especificar ese parametro

In [12]:
df.head(5)

,index,filename,text
0,1,suspicious-document00001.txt,The convening of this body gave rise to much d...
1,2,suspicious-document00002.txt,WOMAN\n\nMAN'S EQUAL.\n\nBY\n\nRev. THOS. WEBS...
2,3,suspicious-document00003.txt,"I do not remember that I was ""very angry"" abou..."
3,4,suspicious-document00004.txt,"Tulaji Angria was dark, inclined to be fat, ..."
4,5,suspicious-document00005.txt,Hiroshima was the primary target of the first ...


In [13]:
class Segmentation:
    
    def __init__(self, text):
        
        self.text = text
        
    def sentSegmentation(self):
        
        sentence = split_into_sentences(self.text)
        
        return list(filter(bool, sentence)) # Elimina oraciones vacías
    
    def paraSegmentation(self):
        
        texto = self.text.split("\n\n")
        
        return list(filter(bool, texto)) # Elimina parrafos vacíos

In [ ]:
SPECIAL_CHARACTERS = []

SPECIAL_CHARACTERS.extend(map(chr, range(0, 32)))
SPECIAL_CHARACTERS.extend(map(chr, range(33, 48)))
SPECIAL_CHARACTERS.extend(map(chr, range(58, 65)))
SPECIAL_CHARACTERS.extend(map(chr, range(91, 97)))
SPECIAL_CHARACTERS.extend(map(chr, range(123, 225)))
SPECIAL_CHARACTERS.extend(map(chr, range(226, 233)))
SPECIAL_CHARACTERS.extend(map(chr, range(234, 237)))
SPECIAL_CHARACTERS.extend(map(chr, range(238, 241)))
SPECIAL_CHARACTERS.extend(map(chr, range(242, 243)))
SPECIAL_CHARACTERS.extend(map(chr, range(244, 250)))
SPECIAL_CHARACTERS.extend(map(chr, range(251, 880)))

In [ ]:
class CleanText():
    
    def __init__(self, text):
        
        self.text = text
    
    def removePatterns(self):
        
        replacements = (
            ("á", "a"),
            ("é", "e"),
            ("í", "i"),
            ("ó", "o"),
            ("ú", "u"),
        )
        
        self.text = str(self.text)
        
        self.clean_text = self.text.lower()
        
        self.clean_text = re.sub(r"\s{2,}", "", self.clean_text)
        
        self.clean_text = re.sub(r"\n", "", self.clean_text)
        
        self.clean_text = re.sub(r"\d+", "", self.clean_text)
        
        self.clean_text = re.sub(r"^\s+", "", self.clean_text)
        
        self.clean_text = re.sub(r"\s+", "", self.clean_text)
        
        for a, b in replacements:
            
            self.clean_text = self.clean_text.replace(a, b).replace(a.upper(), b.upper())
        
        return self.clean_text 

In [ ]:
j = 0

for i, row in enumerate(df.itertuples(), 1):
    
    # Create the SQL query
    query = "INSERT INTO documento (id_documento, filename, text) VALUES (?, ?, ?)"
    
    # Execute the query
    cursor.execute(query, (i, row.filename, row.text))
    
conn.commit()